In [1]:
import sys
from transformers import pipeline
from sklearn.metrics import f1_score
import pandas as pd
import torch
from datasets import  load_dataset
from tqdm import tqdm
tqdm.pandas()

/Users/vetonmatoshi/opt/anaconda3/envs/politmonitor/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sys.path.append('../code/')

In [3]:
from training_data_handler import TrainingDataHandler
from utils import LabelHandler
tdh = TrainingDataHandler()
lh = LabelHandler()

/Users/vetonmatoshi/opt/anaconda3/envs/politmonitor/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
#tdh = TrainingDataHandler()
#classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

print(device)

classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7", device=device)

mps


In [10]:
#Loading the dataset
language = 'all'
tdh.get_training_data(language=language, affair_text_scope=['zh','ch'], affair_attachment_category='all', running_mode='experimental')
test_dataset = tdh.training_data_df[tdh.training_data_df.split=='test']

In [11]:
# Inserting columns for candidate labels
test_dataset['candidate_labels_zero_shot'] = test_dataset.language.apply(lh.get_all_labels_per_language)


/var/folders/w1/wqc21nmn2556z5pqs1_g_5000000gn/T/ipykernel_22459/158922645.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['candidate_labels_zero_shot'] = test_dataset.language.apply(lh.get_all_labels_per_language)


In [12]:
def classify(sequence_to_classify, label_list=None):
    
    if label_list is None:
        label_list = list(tdh.label2id.keys())
    
    output = classifier(sequence_to_classify, label_list, multi_label=True)
    labels_scores = list(zip(output["labels"], output["scores"]))
    labels_scores = [res[0] for res in labels_scores if res[1]>0.5]
    return labels_scores
    


In [24]:
classify('Infolge der geänderten Gesetzgebung betreffend die Quellenbesteuerung des Einkommens aus Erwerbstätigkeit wird in Zukunft auch Grenzgängerinnen und Grenzgängern als "Quasi-Ansässigen" eine ordentliche Veranlagung offenstehen. Diese Änderung gründet bekanntermassen auf einem Urteil des Bundesgerichtes von 2010. Offensichtlich bestand jedoch weder Eile noch besondere Notwendigkeit, sich anzupassen, denn die angebliche "Nichtkonformität" besteht nun schon seit Jahren, ohne dass dies irgendwelche Folgen gehabt hätte.Die neue Regelung hat für Kantone mit vielen Grenzgängerinnen und Grenzgängern, insbesondere für den Kanton Tessin, äusserst negative Auswirkungen: einerseits geringere Steuereinnahmen (aufgrund der Abzüge, die die ordentlich veranlagten Grenzgängerinnen und Grenzgänger geltend machen können), andererseits der höhere Verwaltungsaufwand (für die Prüfung der geltend gemachten Abzüge), wofür neues Personal eingestellt werden müsste.Ich frage den Bundesrat:Beabsichtigt er, wie dies im Übrigen in verschiedenen Berichten angekündigt wird, für die ordentliche Veranlagung an der Einkommensschwelle von 120 000 Franken pro Jahr festzuhalten, um so die Zahl der Grenzgängerinnen und Grenzgänger zu begrenzen, die die Möglichkeit einer ordentlichen Veranlagung haben?')

['Finanzwesen',
 'Soziale Frage',
 'Diverses',
 'Beschäftigung und Arbeit',
 'Grundversorgung',
 'Recht allgemein',
 'Forschung',
 'Alter',
 'Politisches System',
 'Wirtschaft']

In [23]:
classify('veranlagung, grenzgänger, grenzgängerinn, quellenbesteuerung, quasi, tessin, eile, nichtkonformität, einkommensschwelle, ansässige')

['Verkehr', 'Alter', 'Migration und Asyl', 'Landwirtschaft', 'Grundversorgung']

In [13]:
# Inserting the zero-shot-results

test_dataset['zero_shot_results'] = test_dataset.progress_apply(lambda x: classify(x['text'], x['candidate_labels_zero_shot']), axis=1)

100%|████████████████████████████████████████| 3365/3365 [11:43:27<00:00, 12.54s/it]
/var/folders/w1/wqc21nmn2556z5pqs1_g_5000000gn/T/ipykernel_22459/1911696141.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['zero_shot_results'] = test_dataset.progress_apply(lambda x: classify(x['text'], x['candidate_labels_zero_shot']), axis=1)


In [14]:
#test_dataset.to_json('results_zero_shot_short.jsonl', lines=True, orient='records', force_ascii=False)

In [22]:
for item in test_dataset.to_dict(orient="records"):
    text = item['text']
    zero_shot_results = set(item['zero_shot_results'])
    affair_topic_codes_as_labels = set(item['affair_topic_codes_as_labels'])
    intersection_labels = zero_shot_results.intersection(affair_topic_codes_as_labels)
    if len(intersection_labels)>0:
        print(intersection_labels)
        #print(affair_topic_codes_as_labels)
        print('+++++++++++++++++++')
        print(text)
        print('\n###################\n')
    

{'Landwirtschaft'}
+++++++++++++++++++
Der Bundesrat ist gegenwärtig daran, mit der EU die Zweckmässigkeit und den Verhandlungsrahmen für allfällige Bilaterale III zu prüfen. Neben dem institutionellen Rahmen könnten auch die Dossiers Reach, Agrarfreihandel, Elektrizität und Steuerfragen betroffen sein.Die Verhandlungen mit der EU zu einem Freihandelsabkommen im Agrarbereich sind seit mehr als zwei Jahren im Gange. Das Mandat umfasst den Marktzugang für Agrarprodukte, die Produktesicherheit, die öffentliche Gesundheit und die Lebensmittelsicherheit. Diese Verhandlungen sind aufgrund des Widerstands im Parlament und der Evaluation von Bilateralen III praktisch eingefroren.Im Landwirtschaftsbereich würde der Freihandel zum Verschwinden von Tausenden von Stellen im Agrar- und Lebensmittelsektor führen. Auf die Landwirtschaft kommt wohl zudem mittelfristig ein WTO-Abkommen zu, was an sich schon eine grosse Herausforderung ist. Solche Freihandelsabkommen würden die Nachhaltigkeit der Schwei

In [15]:
test_dataset.columns.tolist()

['affair_text_scope',
 'affair_text_srcid',
 'affair_text_date_staged',
 'text_name_de',
 'text_name_fr',
 'text_name_it',
 'text_fr',
 'text_it',
 'source',
 'affair_attachment_title',
 'affair_attachment_name',
 'affair_attachment_type',
 'affair_attachment_link',
 'affair_attachment_category',
 'id_for_merge',
 'affair_scope',
 'affair_srcid',
 'affair_key',
 'affair_closed',
 'affair_date_staged',
 'affair_date_updated',
 'affair_deposit_date',
 'affair_deposit_legislativeperiod',
 'affair_deposit_session',
 'affair_state_id',
 'state_name_de',
 'state_name_fr',
 'title_fr',
 'title_it',
 'type_name_de',
 'type_name_fr',
 'type_name_it',
 'affairtype_curiavista_de',
 'affairtype_curiavista_fr',
 'affairtype_curiavista_it',
 'deposit_council_name_de',
 'deposit_council_abbreviation_de',
 'deposit_council_abbreviation_fr',
 'deposit_council_name_fr',
 'deposit_council_abbreviation_it',
 'deposit_council_name_it',
 'councillor_name',
 'fraction_name',
 'commitee_name',
 'author_canton